In [78]:
%glue_version 3.0
%idle_timeout 60
%connections bi-deltalake-connector
%%configure 
{
  "--conf": "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog",
  #"--extra-py-files": "/tmp/etl-delta-core_2.12-1.0.0.jar" # for custom connector
  "--extra-py-files": "/tmp/delta-core_2.12-1.0.0.jar" # for marketplace connector
}

Setting Glue version to: 3.0
Current idle_timeout is 60 minutes.
idle_timeout has been set to 60 minutes.
Connections to be included:
bi-deltalake-connector
The following configurations have been updated: {'--conf': 'spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog', '--extra-py-files': '/tmp/delta-core_2.12-1.0.0.jar'}


In [73]:
!ls -lrt /tmp

total 2144
-rw-r--r-- 1 root         root   29641 Sep 16 14:41 jupyter_lab_config.py
drwxr-xr-x 3 jupyter-user root    4096 Sep 16 14:47 npm-49-27268cf9
drwxr-xr-x 2 jupyter-user root    4096 Sep 16 14:48 yarn--1663339687276-0.6534362807951573
drwxr-xr-x 2 jupyter-user root    4096 Sep 16 14:48 yarn--1663339687276-0.0031633243502735553
-rw-r--r-- 1 jupyter-user root     653 Sep 16 14:48 core-js-banners
drwxr-xr-x 2 jupyter-user root    4096 Sep 16 14:48 yarn--1663339688038-0.28055851563795686
drwxr-xr-x 2 jupyter-user root    4096 Sep 16 14:48 yarn--1663339691761-0.5388782314584746
drwxr-xr-x 3 jupyter-user root    4096 Sep 16 14:48 v8-compile-cache-10000
-rw-r--r-- 1 jupyter-user root 2109245 Dec 14 17:06 delta-core_2.12-1.0.0.jar
-rw-r--r-- 1 jupyter-user root   23044 Dec 14 17:19 Untitled.ipynb


In [55]:
!curl -O --output-dir /tmp/  https://repo1.maven.org/maven2/io/delta/delta-core_2.12/1.0.0/delta-core_2.12-1.0.0.jar


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2059k  100 2059k    0     0  35.3M      0 --:--:-- --:--:-- --:--:-- 35.9M


In [7]:
bucket_name = "ecosia-bi-demo"
bucket_prefix = "delta"
database_name = "demo"
table_name = "products"
table_prefix = f"{bucket_prefix}/{database_name}/{table_name}"
table_location = f"s3://{bucket_name}/{table_prefix}"

In [8]:
table_location

's3://ecosia-bi-demo/delta/demo/products'


In [9]:
table_prefix

'delta/demo/products'


In [80]:
%status	

Session ID: 6a57a7b1-a135-40e4-9327-dddb8feab8c7
Status: READY
Duration: 50.620792 seconds
Role: arn:aws:iam::178965524635:role/service-role/AWSGlueServiceRole-AWSGlueBIEnrichRole
CreatedOn: 2022-12-14 17:20:00.871000+00:00
GlueVersion: 3.0
Worker Type: G.1X
Number of Workers: 5
Region: eu-central-1
Connections: ['bi-deltalake-connector']
Applying the following default arguments:
--glue_kernel_version 0.35
--enable-glue-datacatalog true
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog
--extra-py-files /tmp/delta-core_2.12-1.0.0.jar
Arguments Passed: ['--glue_kernel_version: 0.35', '--enable-glue-datacatalog: true', '--conf: spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog', '--extra-py-files: /tmp/delta-core_2.12-1.0.0.jar']


## Initialize SparkSession

In [10]:
from pyspark.sql import SparkSession
spark.stop()
spark = SparkSession.builder.getOrCreate()

In [11]:
from pyspark.sql import Row
import time

ut = time.time()

product = [
    {'product_id': '00001', 'product_name': 'Heater', 'price': 250, 'category': 'Electronics', 'updated_at': ut},
    {'product_id': '00002', 'product_name': 'Thermostat', 'price': 400, 'category': 'Electronics', 'updated_at': ut},
    {'product_id': '00003', 'product_name': 'Television', 'price': 600, 'category': 'Electronics', 'updated_at': ut},
    {'product_id': '00004', 'product_name': 'Blender', 'price': 100, 'category': 'Electronics', 'updated_at': ut},
    {'product_id': '00005', 'product_name': 'USB charger', 'price': 50, 'category': 'Electronics', 'updated_at': ut}
]

df_products = spark.createDataFrame(Row(**x) for x in product)

In [12]:
df_products.show()

+----------+------------+-----+-----------+--------------------+
|product_id|product_name|price|   category|          updated_at|
+----------+------------+-----+-----------+--------------------+
|     00001|      Heater|  250|Electronics|1.6710388617707026E9|
|     00002|  Thermostat|  400|Electronics|1.6710388617707026E9|
|     00003|  Television|  600|Electronics|1.6710388617707026E9|
|     00004|     Blender|  100|Electronics|1.6710388617707026E9|
|     00005| USB charger|   50|Electronics|1.6710388617707026E9|
+----------+------------+-----+-----------+--------------------+


In [13]:
%%sql
CREATE DATABASE demo LOCATION 's3://ecosia-bi-demo/delta/demo/';

++
||
++
++


In [14]:
df_products.write.format("delta"). \
  mode("overwrite"). \
  save(table_location)

In [15]:
# create table in metastore
query = f"""
CREATE TABLE {database_name}.{table_name}
USING delta
LOCATION '{table_location}'
"""
spark.sql(query)

DataFrame[]


In [16]:
%%sql
USE demo

++
||
++
++


In [17]:
%%sql
SHOW TABLES

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|    demo| products|      false|
+--------+---------+-----------+


## Read from Delta Lake table

In [18]:
%%sql #Read table from metastore
SELECT * FROM demo.products

+----------+------------+-----+-----------+--------------------+
|product_id|product_name|price|   category|          updated_at|
+----------+------------+-----+-----------+--------------------+
|     00005| USB charger|   50|Electronics|1.6710388617707026E9|
|     00003|  Television|  600|Electronics|1.6710388617707026E9|
|     00002|  Thermostat|  400|Electronics|1.6710388617707026E9|
|     00004|     Blender|  100|Electronics|1.6710388617707026E9|
|     00001|      Heater|  250|Electronics|1.6710388617707026E9|
+----------+------------+-----+-----------+--------------------+


## Stop Session

In [71]:
%stop_session

Stopping session: 6e616d08-0bca-4ea8-a9d7-7dbf82155733
Stopped session.
